In [2]:
import pandas as pd
import numpy as np
import pylab as pl
import mpl_toolkits.basemap as bm
import csv
import os
from skimage.transform import resize, rescale
from skimage.color import rgb2gray
import json
import keras
from sklearn.cross_validation import train_test_split
from keras.layers import Dropout 
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import theano
import skimage.io as io
from skimage.io import imread

Using Theano backend.
Using gpu device 0: GeForce GT 740M (CNMeM is disabled, cuDNN not available)


In [ ]:
# Функции для перевода символьного представления лэйбла класса в числовое и обратно

In [3]:
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)

In [ ]:
#Функция обучения классификатора

In [70]:
def train_classifier():
    #Выкачиваем данные из csv-файла и переводим в numpy представление 
    TRAINING_SET_PATH = "trainLabels.csv"
    df_train = pd.read_csv(TRAINING_SET_PATH, sep=",", header=0)
    train_labels_pd =df_train[u'Class'] 
    train_labels =  train_labels_pd.as_matrix()

    TRAIN_IMG_PATH = 'train'

    n_samples = train_labels.shape[0]
#train_Features - массив картинок тренировочной выборки
    train_Features = np.empty((n_samples,32,32), float)

    for i in range(1,n_samples):
        img = io.imread(TRAIN_IMG_PATH  + '/' + str(i) + '.Bmp')
        #Так как часть картинок - цветная, а часть - черно/белая, сделаем все изображения черно-белыми  
        img = rgb2gray(img)
        # приводим все изображения к одному размеру. Размер изображения - компромисс между точностью классификации и 
        # временем работы алгоритма
        img = resize(img,(32,32))

        train_Features[i-1] = img
    # переводим лэйблы классов в числовое представление
    train_labels_new = np.zeros((n_samples,62), int)
    for i in range(n_samples):

        k = label2int(train_labels[i]) 
        train_labels_new[i][k] = 1

    n_classes = len(set(train_labels))
# Объект ImageDataGenerator используется для размножения данных (поворот, смещение изображений тренировочной выборки)

    datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.15,
            height_shift_range = 0.15,
            shear_range = 0.4,
            zoom_range = 0.3,                    
            channel_shift_range = 0.1)
            

    # Задаем количество эпох, равно 100. Хотелось бы усовершенствовать алгоритм, например, увеличив количество эпох и добавив
    # добавив постепенное уменьшение learning rate и остановку в момент, когда loss достигает плато, однако в связи с большим
    # временем обучения и недостатком мощности моего компьютера, я решила ограничиваться более простым вариантом
    nb_epoch = 100
  
    train_Features = train_Features.reshape(-1, 1, 32, 32)
    # делим выборку на тренировочную и тестовую
    X_train, X_val, y_train, y_val = train_test_split(train_Features, train_labels_new, test_size=0.2, random_state=42)
   
    model = Sequential()
    #Архитектура сети : (Convo - Convo - Pooling) х 2 -> (Convo) х 3 -> Pooling -> (FC) х 3
    # идея архитектуры взята из лекций стэндфорда (http://cs231n.github.io/convolutional-networks/) и изменена для конкретной
    # задачи
    model.add(Convolution2D(128, 3, 3, border_mode='same', init='he_normal', activation = 'relu', input_shape=(1, 32, 32)))
    model.add(Convolution2D(128, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(256, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(256, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))
    model.add(Convolution2D(512, 3, 3, border_mode='same', init='he_normal', activation = 'relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096, init='he_normal', activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, init='he_normal', activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(62, init='he_normal', activation = 'softmax'))
    # Задаем параметры модели
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adadelta',  
                  metrics=["accuracy"])
    # Обучаем модель на тренировочной выборки, оценивая качество на валидационной. При этом данные размножаются в процессе
    # обучения
    model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),
                        samples_per_epoch=len(X_train),
                        nb_epoch=nb_epoch, 
                        validation_data=(X_val, y_val),
                        verbose=1)


    return model


In [ ]:
# функция классификации 

In [4]:
def classify(model):
    TEST_IMG_PATH = 'test'
    # n - количество изображений тестовой выборки
    n = 12504 - 6284 
    # приводим изображения тестовой выборки к тому же виду, что и тренировочные изображения
    Images = np.empty((n,32,32), float)
    for i in range(6284, 12504):
        img = io.imread(TEST_IMG_PATH + '/' + str(i) + '.Bmp')
        img = rgb2gray(img)
        img = resize(img,(32,32))
        Images[i - 6284] = img

    Images = Images.reshape(-1,1,32,32)
# получаем предсказания классов, к которым относятся изображения тестовой выборки 
    labels = model.predict_classes(Images)
# переводим метки классов в символьное представоение
    vInt2label = np.vectorize(int2label)
    Y_test_pred = vInt2label(labels)
#сохраняем результаты в формате, требуемым системой kaggle
    path = 'predictions'
    np.savetxt(path+"/avg_pred.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')

In [ ]:
# вызываем функции обучения классификатора и классификации изображений
model = train_classifier()
classify(model)